In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from functools import reduce

Create the metadata

In [2]:
IMREP_df = pd.read_csv("../summary_data/original/IMREP/IMREP_TRB_merged_extracted_features.csv")
TRUST4_df = pd.read_csv("../summary_data/original/TRUST4/TRUST4_TRB_merged_extracted_features.csv")
MIXCR_df = pd.read_csv("../summary_data/original/MIXCR/MIXCR_TRB_merged_extracted_features.csv")
TCR_df = pd.read_csv("../summary_data/original/TCR_Seq/TCR_merged_extracted_features.csv")

In [3]:
# Select rows with nReads greater than 1
IMREP_df = IMREP_df[IMREP_df['nReads'] > 1]
TRUST4_df = TRUST4_df[TRUST4_df['nReads'] > 1]
MIXCR_df = MIXCR_df[MIXCR_df['nReads'] > 1]
TCR_df = TCR_df[TCR_df['nReads'] > 1]

In [4]:
# Rename nReads and frequencies columns according to tool for proper merging
IMREP_df = IMREP_df.rename(columns={"nReads": "nReads_IMREP"})
TRUST4_df = TRUST4_df.rename(columns={"nReads": "nReads_TRUST4"})
MIXCR_df = MIXCR_df.rename(columns={"nReads": "nReads_MIXCR"})
TCR_df = TCR_df.rename(columns={"nReads": "nReads_TCR"})

In [5]:
# Merge dataframes based on two key combination: Sample and CDR3. Outer join ensures no data is lost for instances that do not have overlap
merge_IMREP_TRUST4 = pd.merge(IMREP_df, TRUST4_df, how='outer', on=['Sample', 'CDR3']).fillna(0)
merge_IMREP_TRUST4_MIXCR = pd.merge(MIXCR_df, merge_IMREP_TRUST4, how='outer', on=['Sample', 'CDR3']).fillna(0)
merge_complete = pd.merge(TCR_df, merge_IMREP_TRUST4_MIXCR, how='outer', on=['Sample', 'CDR3']).fillna(0)

In [6]:
# Select monoclonal repertoires
monoclonal = merge_complete.loc[merge_complete['Sample'].isin(['sample01','sample02','sample03'])]

In [7]:
# Calculate total number of reads in each sample
total_reads = monoclonal[['Sample','nReads_TCR','nReads_MIXCR','nReads_IMREP','nReads_TRUST4']].groupby('Sample').sum().rename(columns={'nReads_TCR':'total_reads_TCR','nReads_MIXCR':'total_reads_MIXCR','nReads_IMREP':'total_reads_IMREP','nReads_TRUST4':'total_reads_TRUST4'})

In [8]:
# Merge dataframes 
merge = pd.merge(monoclonal, total_reads, how='outer', on=['Sample']).fillna(0)

# Calculate frequency of CDR3 reads with respect to CDR3s that occur more than once
merge['frequency_TCR'] = merge['nReads_TCR'] / (merge['total_reads_TCR'] * 1.0)
merge['frequency_MIXCR'] = merge['nReads_MIXCR'] / (merge['total_reads_MIXCR'] * 1.0)
merge['frequency_IMREP'] = merge['nReads_IMREP'] / (merge['total_reads_IMREP'] * 1.0)
merge['frequency_TRUST4'] = merge['nReads_TRUST4'] / (merge['total_reads_TRUST4'] * 1.0)  
merge.fillna(0, inplace=True)

In [9]:
diversity_original = merge[['Sample','CDR3','frequency_TCR']]
diversity_original = diversity_original[diversity_original.frequency_TCR != 0]
diversity_original['shannon_index_TCR'] = -(diversity_original['frequency_TCR']*np.log(diversity_original['frequency_TCR']))
shannon_original = diversity_original.groupby(['Sample']).agg({'shannon_index_TCR':'sum'}).reset_index().rename(columns={'':"shannon_index_TCR"})
shannon_original = shannon_original[['Sample', 'shannon_index_TCR']]
shannon_original

Sample  shannon_index_TCR
0  sample01           1.072675
1  sample02           1.001202
2  sample03           0.960516

In [10]:
# MIXCR
MIXCR = merge[['Sample','CDR3','nReads_MIXCR','total_reads_MIXCR','frequency_MIXCR','frequency_TCR']]
MIXCR.rename(columns={'nReads_MIXCR': 'nReads_tool', 'total_reads_MIXCR': 'total_reads_tool','frequency_MIXCR':'frequency_tool'}, inplace=True)
MIXCR.loc[:,'tool'] = 'MIXCR'

# IMREP
IMREP = merge[['Sample','CDR3','nReads_IMREP','total_reads_IMREP','frequency_IMREP','frequency_TCR']]
IMREP.rename(columns={'nReads_IMREP': 'nReads_tool', 'total_reads_IMREP': 'total_reads_tool','frequency_IMREP':'frequency_tool'}, inplace=True)
IMREP.loc[:,'tool'] = 'IMREP'

# TRUST4
TRUST4 = merge[['Sample','CDR3','nReads_TRUST4','total_reads_TRUST4','frequency_TRUST4','frequency_TCR']]
TRUST4.rename(columns={'nReads_TRUST4': 'nReads_tool', 'total_reads_TRUST4': 'total_reads_tool','frequency_TRUST4':'frequency_tool'}, inplace=True)
TRUST4.loc[:,'tool'] = 'TRUST4'

df = pd.concat([MIXCR,IMREP,TRUST4], sort=False)
df

/Users/keruipeng/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)
/Users/keruipeng/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/keruipeng/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

Sample               CDR3  nReads_tool  total_reads_tool  \
0      sample01     CAWRGDTAQQPQHF          0.0          132472.0   
1      sample01      CASSRDSPETQYF          0.0          132472.0   
2      sample01  CASSYSGRALGTGELFF          0.0          132472.0   
3      sample01    CASSPDGGLRSPLHF          0.0          132472.0   
4      sample01  CASTPRGTVTSNQPQHF          3.0          132472.0   
...         ...                ...          ...               ...   
28838  sample03    CSFSHGDSRYNEQFF          2.0           99723.0   
28839  sample03   CSVETGLALDTDTQYF          4.0           99723.0   
28840  sample03     CASSAGQGNIGYTF          4.0           99723.0   
28841  sample03       CASSPPGRGYTF          2.0           99723.0   
28842  sample03    CSAPSPAGTYNEQFF          3.0           99723.0   

       frequency_tool  frequency_TCR    tool  
0            0.000000       0.000022   MIXCR  
1            0.000000       0.000022   MIXCR  
2            0.000000       0.000022   MIXCR  
3            0.000000       0.000022   MIXCR  
4            0.000023       0.000044   MIXCR  
...               ...            ...     ...  
28838        0.000020       0.000000  TRUST4  
28839        0.000040       0.000000  TRUST4  
28840        0.000040       0.000000  TRUST4  
28841        0.000020       0.000000  TRUST4  
28842        0.000030       0.000000  TRUST4  

[86529 rows x 7 columns]

Reduce the number of reads from MIXCR, IMREP, and TRUST4 to 28,000

In [11]:
samples = ['sample01','sample02','sample03']

df1 = pd.DataFrame()
for sample in samples: 
    sample = df.loc[df['Sample']==sample]
    sample.loc[:,'new_nReads_tool'] = 10000*sample['frequency_tool']
    sample.loc[:,'new_nReads_tool'] = sample['new_nReads_tool'].round().astype(int)
    sample.loc[:,'new_total_reads_tool'] = sample['new_nReads_tool'].sum()
    sample.loc[:,'new_frequency_tool'] = sample['new_nReads_tool'] / (sample['new_total_reads_tool'] * 1.0)
    tool_count = sample[sample.new_nReads_tool != 0]
    clonotype_count = tool_count.groupby(['tool'], sort=False).size().reset_index(name='clonotype_count_tool')
    sample = pd.merge(sample, clonotype_count, on=['tool'])
    df1 =pd.concat([df1, sample])
    
df1.loc[:,'est_reads'] = 28000

In [12]:
tools = ['MIXCR','IMREP','TRUST4']
shannon1 = pd.DataFrame()
for tool in tools:
    subsample = df1.loc[df1['tool']==tool]
    diversity = subsample[subsample.new_frequency_tool != 0]
    diversity.loc[:,'shannon_index_tool'] = -(diversity['new_frequency_tool']*np.log(diversity['new_frequency_tool']))
    shannon = diversity.groupby(['Sample']).agg({'shannon_index_tool':'sum'}).reset_index().rename(columns={'':"shannon_index_tool"})
    shannon = shannon[['Sample', 'shannon_index_tool']]
    shannon.loc[:,'tool'] = tool
    new_reads = df1[['Sample','tool','new_total_reads_tool','est_reads']].drop_duplicates(keep='first', inplace=False)
    shannon = pd.merge(shannon, new_reads, how='left', on=['Sample','tool'])
    shannon1 =pd.concat([shannon1, shannon])

Reduce the number of reads from MIXCR, IMREP, and TRUST4 to 13,500

In [13]:
samples = ['sample01','sample02','sample03']

df2 = pd.DataFrame()
for sample in samples: 
    sample = df.loc[df['Sample']==sample]
    sample.loc[:,'new_nReads_tool'] = 5000*sample['frequency_tool']
    sample.loc[:,'new_nReads_tool'] = sample['new_nReads_tool'].round().astype(int)
    sample.loc[:,'new_total_reads_tool'] = sample['new_nReads_tool'].sum()
    sample.loc[:,'new_frequency_tool'] = sample['new_nReads_tool'] / (sample['new_total_reads_tool'] * 1.0)
    tool_count = sample[sample.new_nReads_tool != 0]
    clonotype_count = tool_count.groupby(['tool'], sort=False).size().reset_index(name='clonotype_count_tool')
    sample = pd.merge(sample, clonotype_count, on=['tool'])
    df2 =pd.concat([df2, sample])
df2.loc[:,'est_reads'] = 13500

In [14]:
tools = ['MIXCR','IMREP','TRUST4']
shannon2 = pd.DataFrame()
for tool in tools:
    subsample = df2.loc[df2['tool']==tool]
    diversity = subsample[subsample.new_frequency_tool != 0]
    diversity.loc[:,'shannon_index_tool'] = -(diversity['new_frequency_tool']*np.log(diversity['new_frequency_tool']))
    shannon = diversity.groupby(['Sample']).agg({'shannon_index_tool':'sum'}).reset_index().rename(columns={'':"shannon_index_tool"})
    shannon = shannon[['Sample', 'shannon_index_tool']]
    shannon.loc[:,'tool'] = tool
    new_reads = df2[['Sample','tool','new_total_reads_tool','est_reads']].drop_duplicates(keep='first', inplace=False)
    shannon = pd.merge(shannon, new_reads, how='left', on=['Sample','tool'])
    shannon2 =pd.concat([shannon2, shannon])

Reduce the number of reads from MIXCR, IMREP, and TRUST4 to 2600

In [15]:
samples = ['sample01','sample02','sample03']

df3 = pd.DataFrame()
for sample in samples: 
    sample = df.loc[df['Sample']==sample]
    sample.loc[:,'new_nReads_tool'] = 1000*sample['frequency_tool']
    sample.loc[:,'new_nReads_tool'] = sample['new_nReads_tool'].round().astype(int)
    sample.loc[:,'new_total_reads_tool'] = sample['new_nReads_tool'].sum()
    sample.loc[:,'new_frequency_tool'] = sample['new_nReads_tool'] / (sample['new_total_reads_tool'] * 1.0)
    tool_count = sample[sample.new_nReads_tool != 0]
    clonotype_count = tool_count.groupby(['tool'], sort=False).size().reset_index(name='clonotype_count_tool')
    sample = pd.merge(sample, clonotype_count, on=['tool'])
    df3 =pd.concat([df3, sample])
df3.loc[:,'est_reads'] = 2600

In [16]:
tools = ['MIXCR','IMREP','TRUST4']
shannon3 = pd.DataFrame()
for tool in tools:
    subsample = df3.loc[df3['tool']==tool]
    diversity = subsample[subsample.new_frequency_tool != 0]
    diversity.loc[:,'shannon_index_tool'] = -(diversity['new_frequency_tool']*np.log(diversity['new_frequency_tool']))
    shannon = diversity.groupby(['Sample']).agg({'shannon_index_tool':'sum'}).reset_index().rename(columns={'':"shannon_index_tool"})
    shannon = shannon[['Sample', 'shannon_index_tool']]
    shannon.loc[:,'tool'] = tool
    new_reads = df3[['Sample','tool','new_total_reads_tool','est_reads']].drop_duplicates(keep='first', inplace=False)
    shannon = pd.merge(shannon, new_reads, how='left', on=['Sample','tool'])
    shannon3 =pd.concat([shannon3, shannon])

Reduce the number of reads from MIXCR, IMREP, and TRUST4 to 1300

In [17]:
samples = ['sample01','sample02','sample03']

df4 = pd.DataFrame()
for sample in samples: 
    sample = df.loc[df['Sample']==sample]
    sample.loc[:,'new_nReads_tool'] = 500*sample['frequency_tool']
    sample.loc[:,'new_nReads_tool'] = sample['new_nReads_tool'].round().astype(int)
    sample.loc[:,'new_total_reads_tool'] = sample['new_nReads_tool'].sum()
    sample.loc[:,'new_frequency_tool'] = sample['new_nReads_tool'] / (sample['new_total_reads_tool'] * 1.0)
    tool_count = sample[sample.new_nReads_tool != 0]
    clonotype_count = tool_count.groupby(['tool'], sort=False).size().reset_index(name='clonotype_count_tool')
    sample = pd.merge(sample, clonotype_count, on=['tool'])
    df4 =pd.concat([df4, sample])
df4.loc[:,'est_reads'] = 1300

In [18]:
tools = ['MIXCR','IMREP','TRUST4']
shannon4 = pd.DataFrame()
for tool in tools:
    subsample = df4.loc[df4['tool']==tool]
    diversity = subsample[subsample.new_frequency_tool != 0]
    diversity.loc[:,'shannon_index_tool'] = -(diversity['new_frequency_tool']*np.log(diversity['new_frequency_tool']))
    shannon = diversity.groupby(['Sample']).agg({'shannon_index_tool':'sum'}).reset_index().rename(columns={'':"shannon_index_tool"})
    shannon = shannon[['Sample', 'shannon_index_tool']]
    shannon.loc[:,'tool'] = tool
    new_reads = df4[['Sample','tool','new_total_reads_tool','est_reads']].drop_duplicates(keep='first', inplace=False)
    shannon = pd.merge(shannon, new_reads, how='left', on=['Sample','tool'])
    shannon4 =pd.concat([shannon4, shannon])

Reduce the number of reads from MIXCR, IMREP, and TRUST4 to 310

In [19]:
samples = ['sample01','sample02','sample03']

df5 = pd.DataFrame()
for sample in samples: 
    sample = df.loc[df['Sample']==sample]
    sample.loc[:,'new_nReads_tool'] = 118*sample['frequency_tool']
    sample.loc[:,'new_nReads_tool'] = sample['new_nReads_tool'].round().astype(int)
    sample.loc[:,'new_total_reads_tool'] = sample['new_nReads_tool'].sum()
    sample.loc[:,'new_frequency_tool'] = sample['new_nReads_tool'] / (sample['new_total_reads_tool'] * 1.0)
    tool_count = sample[sample.new_nReads_tool != 0]
    clonotype_count = tool_count.groupby(['tool'], sort=False).size().reset_index(name='clonotype_count_tool')
    sample = pd.merge(sample, clonotype_count, on=['tool'])
    df5 =pd.concat([df5, sample])
df5.loc[:,'est_reads'] = 310

In [20]:
# df_T_cell_poor_monoclonal = df5[['Sample', 'CDR3','frequency_TCR','tool','new_nReads_tool','new_total_reads_tool','new_frequency_tool']]
# df_T_cell_poor_monoclonal.loc[:,'Sample'] = df_T_cell_poor_monoclonal['Sample'].replace({'sample01': 'insilco_sample1', 'sample02': 'insilco_sample2', 'sample03':'insilco_sample3'})
# df_T_cell_poor_monoclonal.loc[:,'tissue_type'] = 'T_cell_poor'
# df_T_cell_poor_monoclonal.loc[:,'repertoire_type'] = 'monoclonal'
# df_T_cell_poor_monoclonal.loc[:,'class'] = 'T_cell_poor_monoclonal'

# MIXCR = df_T_cell_poor_monoclonal.loc[df_T_cell_poor_monoclonal['tool'] == 'MIXCR']
# MIXCR = MIXCR.rename({'new_nReads_tool':'nReads_MIXCR','new_total_reads_tool':'total_reads_MIXCR','new_frequency_tool':'frequency_MIXCR'}, axis=1)
# MIXCR = MIXCR.drop(['tool'], axis=1)

# IMREP = df_T_cell_poor_monoclonal.loc[df_T_cell_poor_monoclonal['tool'] == 'IMREP']
# IMREP = IMREP.rename({'new_nReads_tool':'nReads_IMREP','new_total_reads_tool':'total_reads_IMREP','new_frequency_tool':'frequency_IMREP'}, axis=1)
# IMREP = IMREP.drop(['tool'], axis=1)

# TRUST4 = df_T_cell_poor_monoclonal.loc[df_T_cell_poor_monoclonal['tool'] == 'TRUST4']
# TRUST4 = TRUST4.rename({'new_nReads_tool':'nReads_TRUST4','new_total_reads_tool':'total_reads_TRUST4','new_frequency_tool':'frequency_TRUST4'}, axis=1)
# TRUST4 = TRUST4.drop(['tool'], axis=1)

# dfs = [MIXCR,IMREP,TRUST4]
# df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Sample','CDR3','frequency_TCR','tissue_type','repertoire_type','class'],how='outer'), dfs)
# df_merged

In [21]:
tools = ['MIXCR','IMREP','TRUST4']
shannon5 = pd.DataFrame()
for tool in tools:
    subsample = df5.loc[df5['tool']==tool]
    diversity = subsample[subsample.new_frequency_tool != 0]
    diversity.loc[:,'shannon_index_tool'] = -(diversity['new_frequency_tool']*np.log(diversity['new_frequency_tool']))
    shannon = diversity.groupby(['Sample']).agg({'shannon_index_tool':'sum'}).reset_index().rename(columns={'':"shannon_index_tool"})
    shannon = shannon[['Sample', 'shannon_index_tool']]
    shannon.loc[:,'tool'] = tool
    new_reads = df5[['Sample','tool','new_total_reads_tool','est_reads']].drop_duplicates(keep='first', inplace=False)
    shannon = pd.merge(shannon, new_reads, how='left', on=['Sample','tool'])
    shannon5 =pd.concat([shannon5, shannon])

In [22]:
subsample_combined = pd.concat([df1, df2, df3, df4, df5])
clonotype_count = subsample_combined[['Sample','clonotype_count_tool','tool','est_reads']].drop_duplicates(keep='first', inplace=False)

In [23]:
subsample_combined = subsample_combined.drop(['clonotype_count_tool'], axis=1)
subsample_combined.to_csv('../summary_data/subsample/subsample_complete.csv', index=False)

In [24]:
shannon_combined = pd.concat([shannon1, shannon2, shannon3, shannon4, shannon5])
diversity_combined = pd.merge(shannon_original, shannon_combined, how='outer', on=['Sample'])
diversity_combined = pd.merge(diversity_combined, clonotype_count, how='outer', on=['Sample','tool','est_reads'])
diversity_combined['absolute_error'] = np.abs(diversity_combined['shannon_index_tool'] - diversity_combined['shannon_index_TCR'])
diversity_combined

Sample  shannon_index_TCR  shannon_index_tool    tool  \
0   sample01           1.072675            0.448437   MIXCR   
1   sample01           1.072675            0.547504   IMREP   
2   sample01           1.072675            0.518910  TRUST4   
3   sample01           1.072675            0.389090   MIXCR   
4   sample01           1.072675            0.459795   IMREP   
5   sample01           1.072675            0.461263  TRUST4   
6   sample01           1.072675            0.372418   MIXCR   
7   sample01           1.072675            0.406998   IMREP   
8   sample01           1.072675            0.396686  TRUST4   
9   sample01           1.072675            0.366791   MIXCR   
10  sample01           1.072675            0.386767   IMREP   
11  sample01           1.072675            0.376255  TRUST4   
12  sample01           1.072675            0.366924   MIXCR   
13  sample01           1.072675            0.365766   IMREP   
14  sample01           1.072675            0.365766  TRUST4   
15  sample02           1.001202            0.526335   MIXCR   
16  sample02           1.001202            1.009263   IMREP   
17  sample02           1.001202            0.717945  TRUST4   
18  sample02           1.001202            0.412935   MIXCR   
19  sample02           1.001202            0.607380   IMREP   
20  sample02           1.001202            0.557980  TRUST4   
21  sample02           1.001202            0.370954   MIXCR   
22  sample02           1.001202            0.412965   IMREP   
23  sample02           1.001202            0.444505  TRUST4   
24  sample02           1.001202            0.367854   MIXCR   
25  sample02           1.001202            0.391210   IMREP   
26  sample02           1.001202            0.421393  TRUST4   
27  sample02           1.001202            0.367878   MIXCR   
28  sample02           1.001202            0.363126   IMREP   
29  sample02           1.001202            0.364714  TRUST4   
30  sample03           0.960516            0.564798   MIXCR   
31  sample03           0.960516            0.701097   IMREP   
32  sample03           0.960516            0.637789  TRUST4   
33  sample03           0.960516            0.490193   MIXCR   
34  sample03           0.960516            0.544536   IMREP   
35  sample03           0.960516            0.540777  TRUST4   
36  sample03           0.960516            0.457819   MIXCR   
37  sample03           0.960516            0.450677   IMREP   
38  sample03           0.960516            0.455030  TRUST4   
39  sample03           0.960516            0.457339   MIXCR   
40  sample03           0.960516            0.425738   IMREP   
41  sample03           0.960516            0.431062  TRUST4   
42  sample03           0.960516            0.452014   MIXCR   
43  sample03           0.960516            0.381516   IMREP   
44  sample03           0.960516            0.401539  TRUST4   

    new_total_reads_tool  est_reads  clonotype_count_tool  absolute_error  
0                  28349      28000                   219        0.624237  
1                  28349      28000                   369        0.525171  
2                  28349      28000                   316        0.553765  
3                  13889      13500                    31        0.683585  
4                  13889      13500                    93        0.612880  
5                  13889      13500                   109        0.611412  
6                   2739       2600                     3        0.700256  
7                   2739       2600                    13        0.665677  
8                   2739       2600                    11        0.675989  
9                   1362       1300                     1        0.705884  
10                  1362       1300                     5        0.685907  
11                  1362       1300                     3        0.696420  
12                   319        310                     1        0.705750  
13                   319        310        

In [25]:
diversity_combined.to_csv('../summary_data/subsample/subsample_diversity.csv', index=False)

In [26]:
# diversity_T_cell_poor_monoclonal = diversity_combined.loc[diversity_combined['est_reads']==310]
# diversity_T_cell_poor_monoclonal.loc[:,'Sample'] = diversity_T_cell_poor_monoclonal['Sample'].replace({'sample01': 'insilco_sample1', 'sample02': 'insilco_sample2', 'sample03':'insilco_sample3'})
# diversity_T_cell_poor_monoclonal.loc[:,'tissue_type'] = 'T_cell_poor'
# diversity_T_cell_poor_monoclonal.loc[:,'repertoire_type'] = 'monoclonal'
# diversity_T_cell_poor_monoclonal.loc[:,'class'] = 'T_cell_poor_monoclonal'
# diversity_T_cell_poor_monoclonal = diversity_T_cell_poor_monoclonal.drop(['est_reads'], axis=1)
# diversity_T_cell_poor_monoclonal = diversity_T_cell_poor_monoclonal.drop(['new_total_reads_tool'], axis=1)
# diversity_T_cell_poor_monoclonal